<a href="https://colab.research.google.com/github/harshitadd/CrossSiloFLDemo/blob/master/Multiple_worker_PreVID_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Hellllo")

Hellllo


In [2]:
!pip install --quiet syft pydicom tqdm

     |████████████████████████████████| 378kB 2.9MB/s 
     |████████████████████████████████| 35.3MB 1.3MB/s 
     |████████████████████████████████| 2.1MB 49.7MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 753.4MB 20kB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 204kB 51.4MB/s 
     |████████████████████████████████| 4.0MB 53.4MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 1.8MB 45.2MB/s 
     |████████████████████████████████| 1.3MB 51.2MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERRO

In [3]:
##Importing Mini Batch Data 
import os 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pydicom 
import cv2
import matplotlib.pyplot as plt 

In [0]:
!pip install tqdm

In [4]:
from tqdm import tqdm

dcm_path=os.listdir('/content/drive/My Drive/Fed_Covid/minibatch_large/')
dcm_data={}

alpha = 1.5 # Contrast control (1.0-3.0)
beta = 0 # Brightness control (0-100)

for file in tqdm(dcm_path):
  name = '/content/drive/My Drive/Fed_Covid/minibatch_large/' + file
  temp = pydicom.dcmread(name)
  image = temp.pixel_array
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  adjusted = cv2.resize(image,(64,64))
  dcm_data[file]=adjusted

100%|██████████| 4006/4006 [10:55<00:00,  6.11it/s]


In [40]:
import csv 
labels=[]
with open('/content/drive/My Drive/Fed_Covid/stage_2_train_labels.csv','r') as file:
  reader = csv.reader(file)
  for row in tqdm(reader):
    labels.append(row)

30228it [00:00, 146111.77it/s]


In [0]:
from sklearn.preprocessing import MinMaxScaler 
import numpy as np
scaler = MinMaxScaler()
pid=[]
dicom=[]
label=[]
cid = 0
for PID in labels:
  for key in dcm_data:
    if(key[:-4]==PID[0]):
      l=[]
      for val in dcm_data[key]:
        l.append(scaler.fit_transform(val))
      l = np.reshape(l,(3,64,64))
      dicom.append(l)
      label.append(int(PID[5]))
      

In [42]:
import numpy as np
print(np.shape(dicom[0]))

(3, 64, 64)


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
from torch.utils.data import DataLoader, Dataset

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dicom,label,test_size=0.3)

In [0]:
import pandas as pd 
train_df = pd.DataFrame()
train_df['features']=x_train
train_df['labels']=y_train

In [47]:
train_df

,features,labels
0,"[[[0.018404907975460124, 0.018404907975460124,...",1
1,"[[[0.06060606060606061, 0.06060606060606061, 0...",0
2,"[[[0.006666666666666667, 0.006666666666666667,...",1
3,"[[[0.10370370370370371, 0.10370370370370371, 0...",0
4,"[[[0.7285714285714286, 0.7285714285714286, 0.7...",0
...,...,...
2984,"[[[0.02958579881656804, 0.02958579881656804, 0...",0
2985,"[[[0.0, 0.0, 0.0, 0.6288209606986899, 0.628820...",0
2986,"[[[0.07253886010362695, 0.07253886010362695, 0...",0
2987,"[[[0.012269938650306749, 0.012269938650306749,...",0


In [0]:
test_df = pd.DataFrame()
test_df['features']=x_test
test_df['labels']=y_test

In [49]:
test_df

,features,labels
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01310043668...",0
1,"[[[0.009900990099009901, 0.009900990099009901,...",0
2,"[[[0.006896551724137931, 0.006896551724137931,...",0
3,"[[[0.4233576642335766, 0.4233576642335766, 0.4...",0
4,"[[[0.0, 0.0, 0.0, 0.22302158273381295, 0.22302...",0
...,...,...
1276,"[[[0.06796116504854369, 0.06796116504854369, 0...",0
1277,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05113636363...",0
1278,"[[[0.005555555555555555, 0.005555555555555555,...",0
1279,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",1


In [0]:
class data_maker(Dataset):
  def __init__(self, data):
        self.data = data
        
  def __len__(self):
      return len(self.data)
  
  def __getitem__(self, index):
      target = np.array(self.data['labels'][index])
      data_val = np.array(self.data['features'][index])

      return data_val,target
      
        

In [0]:
train_data = data_maker(train_df)
test_data = data_maker(test_df)

In [0]:
def get_train_data(batch_size):
      train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False)
      return train_loader

In [0]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4)


In [54]:
train_loader= get_train_data(batch_size=4)
n_batches = len(train_loader)
for i, data in enumerate(train_loader, 0):
       inputs, labels = data
       print(np.shape(inputs))
       break   

torch.Size([4, 3, 64, 64])


In [0]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Input - (32,1024,1024)
  
        self.conv1 = nn.Conv2d(3,32, kernel_size=8, stride = 2)  #I/P - 1*32
        self.pool1 = nn.MaxPool2d(kernel_size=4,stride=2,padding=0)  
        self.conv2 = nn.Conv2d(32, 64, 8)
        self.pool2 = nn.MaxPool2d(4,2,padding=0)
        self.fc1 = nn.Linear(256, 64*64)
        self.fc2 = nn.Linear(64*64, 64)
        self.fc3 = nn.Linear(64, 8)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        # Input of 1*1024*1024 going to 32*1024*1024  - 1*64*64 going to 32*64*64
        x = self.pool1(x)
        # Input of 32*1024*1024 going to 32*256*256 - 32*64*64 going to 32*16*16
        x = F.relu(self.conv2(x))
        # Input of 32*256*256 to 64*256*256 - 32*16*16 to 64*16*16
        x = self.pool2(x)
        # Input of 64*256*256 to 64*64*64 - 64*16*16 going to 64*4*4

        x = x.view(-1,64*4)
     
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [0]:
def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [0]:
import time
from torch.autograd import Variable
def trainNet(net, batch_size, n_epochs, learning_rate):
    
    
    #Get training data
    train_loader = get_train_data(batch_size=8)
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
          
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            

            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            

            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data
            total_train_loss += loss_size.data
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [58]:
CNN = Net()
trainNet(CNN.double(), batch_size=8, n_epochs=10
         , learning_rate=0.001)

Epoch 1, 10% 	 train_loss: 0.79 took: 2.56s
Epoch 1, 20% 	 train_loss: 0.53 took: 2.54s
Epoch 1, 30% 	 train_loss: 0.58 took: 2.54s
Epoch 1, 40% 	 train_loss: 0.50 took: 2.52s
Epoch 1, 50% 	 train_loss: 0.44 took: 2.50s
Epoch 1, 60% 	 train_loss: 0.46 took: 2.49s
Epoch 1, 71% 	 train_loss: 0.45 took: 2.49s
Epoch 1, 81% 	 train_loss: 0.47 took: 2.50s
Epoch 1, 91% 	 train_loss: 0.46 took: 2.47s
Epoch 2, 10% 	 train_loss: 0.47 took: 2.50s
Epoch 2, 20% 	 train_loss: 0.43 took: 2.49s
Epoch 2, 30% 	 train_loss: 0.49 took: 2.51s
Epoch 2, 40% 	 train_loss: 0.45 took: 2.50s
Epoch 2, 50% 	 train_loss: 0.40 took: 2.47s
Epoch 2, 60% 	 train_loss: 0.47 took: 2.49s
Epoch 2, 71% 	 train_loss: 0.42 took: 2.46s
Epoch 2, 81% 	 train_loss: 0.46 took: 2.47s
Epoch 2, 91% 	 train_loss: 0.43 took: 2.46s
Epoch 3, 10% 	 train_loss: 0.41 took: 2.51s
Epoch 3, 20% 	 train_loss: 0.43 took: 2.48s
Epoch 3, 30% 	 train_loss: 0.44 took: 2.49s
Epoch 3, 40% 	 train_loss: 0.42 took: 2.47s
Epoch 3, 50% 	 train_loss: 0.37 

In [59]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = CNN(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print('Accuracy of the network on test images: %d %%' % (100 * correct / total))

Accuracy of the network on test images: 79 %


https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html - Test Loader

PYTORCH IMPLEMENTATION


In [0]:
!pip install --quiet syft

In [68]:
import syft as sy 
hook = sy.TorchHook(torch)

In [0]:
hospital = sy.VirtualWorker(hook, id="hospital")  
clinic1 = sy.VirtualWorker(hook, id="clinic1")
clinic2 = sy.VirtualWorker(hook, id="clinic2")
clinic3 = sy.VirtualWorker(hook, id="clinic3")   

In [70]:
print(x_train[0])

[[[0.01840491 0.01840491 0.01840491 ... 0.01840491 0.01840491 0.01840491]
  [0.01840491 0.01840491 0.01840491 ... 0.93865031 0.88957055 0.88957055]
  [0.88957055 0.88957055 0.88957055 ... 0.02453988 0.02453988 0.02453988]
  ...
  [0.30434783 0.30434783 0.27826087 ... 0.55652174 0.4173913  0.4173913 ]
  [0.4173913  0.40869565 0.40869565 ... 0.31304348 0.31304348 0.31304348]
  [0.58119658 0.58119658 0.58119658 ... 0.43589744 0.43589744 0.24786325]]

 [[0.24786325 0.24786325 0.1025641  ... 0.41880342 0.32478632 0.32478632]
  [0.32478632 0.15384615 0.15384615 ... 0.23076923 0.23076923 0.23076923]
  [0.55045872 0.55045872 0.55045872 ... 0.34862385 0.34862385 0.11926606]
  ...
  [0.78421053 0.74210526 0.74210526 ... 0.03157895 0.03157895 0.03157895]
  [0.29319372 0.29319372 0.29319372 ... 0.65968586 0.65968586 0.60209424]
  [0.60209424 0.60209424 0.62303665 ... 0.81675393 0.83769634 0.83769634]]

 [[0.83769634 0.78534031 0.78534031 ... 0.03141361 0.03141361 0.03141361]
  [0.29441624 0.294416

In [0]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [0]:
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)

In [0]:

#define the send_data function 
# data_hospital = x_train[0:80].send(hospital)
# data_target_hospital = y_train[0:80].send(hospital)
#data_clinic = x_train[80:].send(clinic)
#data_target_clinic = y_train[80:].send(clinic)



#define the send_data function
#Worker1
data_hospital = x_train[0:25].send(hospital)
data_target_hospital = y_train[0:25].send(hospital)
#Worker2
data_clinic1 = x_train[25:50].send(clinic1)
data_target_clinic1 = y_train[25:50].send(clinic1)
#Worker3
data_clinic2 = x_train[50:75].send(clinic2)
data_target_clinic2 = y_train[50:75].send(clinic2)
#Worker4
data_clinic3 = x_train[75:].send(clinic3)
data_target_clinic3 = y_train[75:].send(clinic3)

In [0]:
#datasets=[(data_hospital,data_target_hospital),(data_clinic,data_target_clinic)]
datasets=[(data_hospital,data_target_hospital),(data_clinic1,data_target_clinic1),(data_clinic2,data_target_clinic2),(data_clinic3,data_target_clinic3)]

In [0]:
opt = optim.SGD(params=model.parameters(),lr=0.1)

In [0]:
# def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    
    
#     #Optimizer
#     optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
#     return(loss, optimizer)

##Stateful issue fix - https://github.com/OpenMined/PySyft/pull/3179/files

In [0]:
import copy


class Optims:
    """to create a list of optimizer objects"""

    def __init__(self, workers, optim):
        """
        args:
            workers: list of worker ids
            optim: class of pytorch optimizer
        """
        self.optim = optim
        self.workers = workers
        self.optimizers = {}
        for worker in workers:
            self.optimizers[str(worker)] = copy.copy(self.optim)
            self.optimizers[str(worker)].load_state_dict((self.optim).state_dict())

    def get_optim(self, worker):
        """returns optimizer for worker
        args: 
            worker: worker id
        """
        return self.optimizers[str(worker)]

    def count(self):
        """returns the number of optimizer objects"""
        return len(self.workers)


In [0]:
import time
from torch.autograd import Variable
from syft.federated.floptimizer import Optims
#workers = ['hospital', 'clinic']
workers = ['hospital', 'clinic1', 'clinic2', 'clinic3']
def federated_train_net(model):

  # loss, optimizer = createLossAndOptimizer(model)
  training_start_time = time.time()
    

  for epoch in range(10):
    running_loss = 0.0
    print_every = n_batches // 10
    start_time = time.time()
    total_train_loss = 0
    loss = torch.nn.CrossEntropyLoss()
    for _data, _target in datasets:

      ##send model 
      model = model.send(_data.location)
      
      ##Getting stateful optimizer
      optims = Optims(workers, optim=optim.Adam(params=model.parameters(),lr=0.001)) 
      opt = optims.get_optim(_data.location.id)
      opt.zero_grad()
      

      #Forward pass, backward pass, optimize
      outputs = model(_data)
      loss_size = loss(outputs, _target)
      loss_size.backward()
      opt.step()

      #get the model back
      model = model.get()
      
      

      #Print statistics
      print('Training loss for epoch ' + str(epoch) + str(loss_size.get()) + str(_data.location))
      # running_loss =running_loss + loss_size.data.clone().get()
      # total_train_loss =total_train_loss+ loss_size.data
      
      # #Print every 10th batch of an epoch
      # if (i + 1) % (print_every + 1) == 0:
      #     print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
      #             epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
      #     #Reset running loss and time
      #     running_loss = 0.0
      #     start_time = time.time()


      # print('Training loss for epoch ' + str(epoch) + str(loss.get()) + str(_data.location))

In [77]:
CNN = Net()
federated_train_net(CNN.double())

Training loss for epoch 0tensor(2.0531, dtype=torch.float64, requires_grad=True)<VirtualWorker id:hospital #objects:53>
Training loss for epoch 0tensor(1.7824, dtype=torch.float64, requires_grad=True)<VirtualWorker id:clinic1 #objects:35>
Training loss for epoch 0tensor(0.8838, dtype=torch.float64, requires_grad=True)<VirtualWorker id:clinic2 #objects:47>
Training loss for epoch 0tensor(1.0185, dtype=torch.float64, requires_grad=True)<VirtualWorker id:clinic3 #objects:47>
Training loss for epoch 1tensor(0.8264, dtype=torch.float64, requires_grad=True)<VirtualWorker id:hospital #objects:53>
Training loss for epoch 1tensor(0.1992, dtype=torch.float64, requires_grad=True)<VirtualWorker id:clinic1 #objects:35>
Training loss for epoch 1tensor(0.5886, dtype=torch.float64, requires_grad=True)<VirtualWorker id:clinic2 #objects:47>
Training loss for epoch 1tensor(0.8985, dtype=torch.float64, requires_grad=True)<VirtualWorker id:clinic3 #objects:47>
Training loss for epoch 2tensor(0.6418, dtype=

In [78]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = CNN(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the test images: 78 %
